 ## Sankey (gráfico de flujo o trasvase de inforamción) para votaciones

 

### 0. Importaciión de librerias

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from scipy.interpolate import make_interp_spline

### 1. Presentación y codificación de datos

Los datos que se van a utilizar en este ejercicio de visualización son los que facilitar el CIS en su web (www.cis.es/es/detalle-ficha-estudio?idEstudio=14752), resultado del barómetro de septiembre del pasado año. Dicho barómetro corresponde a la encuesta postelectoral de las elecciones generales de julio de 2023. 

El CIS facilita el dataset con las respuestas de la encuesta y el dataset ya codificado. Se van a utilizar el dataset de los datos códificados en primer lugar porque el nombre de las columnas viene codificado y no es la pregunta de la encuesta del barómetro y en segundo lugar, con la idea de llevar transformaciones (si fueresn necesarias) de forma más rápida, en caso de necesitar nimbrar las categorías para una visualizadión se explicará icha codificación extraida del libro de códigos faciltiado también por el organismo público.

In [30]:
df = pd.read_csv('Datos/3420_num.csv', sep=';')

In [31]:
df.shape

(10101, 232)

In [32]:
pd.set_option('display.max_columns', None)
df.head()

,ESTUDIO,REGISTRO,CUES,PROV,MUN,CCAA,CAPITAL,ENTREV,TAMUNI,TIPO_TEL,SEXO,EDAD,A0,P0,P1,ECOPER,ECOESP,PESPANNA1,PESPANNA2,PESPANNA3,PPERSONAL1,PPERSONAL2,PPERSONAL3,PARTICIPACIONG1,RECUVOTOG1,RECUVOTOG1R,RECUERDO1,RECUERDO1R,P7,P7A,P7B,P7B_1,INTENCIONGIND_2,INTENCIONGIND_1,INTENCIONGIND_3,INTENCIONGIND_21,INTENCIONGIND_901,INTENCIONGIND_902,INTENCIONGIND_904,INTENCIONGIND_1601,INTENCIONGIND_1602,INTENCIONGIND_503,INTENCIONGIND_1201,INTENCIONGIND_202,INTENCIONGIND_1501,INTENCIONGIND_1502,INTENCIONGIND_6,INTENCIONGIND_12,INTENCIONGIND_1901,INTENCIONGIND_905,INTENCIONGIND_808,INTENCIONGIND_801,INTENCIONGIND_601,INTENCIONGIND_201,INTENCIONGIND_25,INTENCIONGIND_24,INTENCIONGIND_19,INTENCIONGIND_17,INTENCIONGIND_13,INTENCIONGIND_8995,INTENCIONGIND_8995R,INTENCIONGIND_8996,INTENCIONGIND_9977,INTENCIONGIND_9997,INTENCIONGIND_9998,INTENCIONGIND_9999,P7C_1,P7C_2,P7C_3,P7C_4,P7C_5,P7C_6,P7C_7,P7C_8,P7C_9,P7C_10,P7C_11,P7C_12,P7C_13,P7C_14,P7C_15,P7C_16,P7C_17,P7C_18,P7C_19,P7C_20,P7C_21,P7C_22,P7C_23,P7C_24,P7C_96,P7C_97,P7C_98,P7C_99,P7D,P7E,P8,P8A,INTENCIONGABS,INTENCIONGABSR,P8C_1,P8C_2,P8C_3,P8C_4,P8C_5,P8C_6,P8C_7,P8C_8,P8C_9,P8C_10,P8C_11,P8C_12,P8C_13,P8C_14,P8C_15,P8C_96,P8C_98,P8C_99,PROBVOTO,INTENCIONG,INTENCIONGALTER,SIMPATIA,PREFPTE,ESCIDEOL,PARTICIPACIONG,RECUVOTOG,CERCANIA,ESCUELA,NIVELESTENTREV,RELIGION,PRACTICARELIG6,ECIVIL,SITLAB,RELALAB,CNO11,INGRESHOG,CLASESOCIAL,SINCERIDAD,IA_I501,IA_I502,IA_I503,IA_I504,IA_I505,IA_I506,IA_I507,IA_I508,IA_I509,IA_I510,IA_I511,IA_I512,IA_I513,IA_I514,IA_I515,IA_I516,IA_I517,IA_I518,IA_I519,IA_I520,IA_I521,IA_I522,IA_I523,IA_I524,IA_I525,IA_I526,IA_I528,IA_I529,IA_I530,IA_I531,IA_I532,IA_I533,IA_I534,IA_I535,IA_I536,IA_I537,IA_I538,IA_I539,IA_I540,IA_I541,IA_I542,IA_I543,IA_OBS0,IA_OBS1,IA_OBS2,IA_E1_DIA,IA_E1_MES,IA_E1_ANYO,IA_E2,IA_E3,IA_E4,IA_VALIDA,IA_APLAZADA,IA_MODIFICADA,IA_SUPERVISADA,IA_C2,IA_C2A,IA_C2B_1,IA_C2B_2,IA_C2B_3,IA_C2B_4,IA_C2B_5,IA_C2B_6,IA_C2B_7,IA_C2B_8,IA_C2B_9,IA_C2B_10,IA_C2B_11,IA_C2B_12,IA_C2B_13,IA_C2B_14,IA_C2B_15,IA_C2B_16,IA_C2B_17,IA_C2B_18,IA_C2B_19,IA_C2B_20,IA_C2B_21,IA_C2B_22,IA_C2B_23,IA_C2B_24,IA_C2B_25,IA_C3,IA_C4,INTENCIONGR,INTENCIONGALTERR,VOTOSIMG,RECUVOTOGR,RECUERDO,ESTUDIOS,CNO11R,CLASESUB,PESO,PESOCCAA
0,3420,46,1,47,186,8,1,0,5,2,2,19,1,1,2,2,2,16,97,99,20,97,99,1,2,2,2,2,1,4,2,3.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,2,0.0,98,1,3,0,2,3,4,5,0.0,2,1,1,7,1,5,2,0,0,0,0,0,0,16,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,2,0,45,0,0,0,0,26,7,NaN,NaN,NaN,1,9,23,5,16,1,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1,0,9995,3,7,4,"1,41998","0,93107"
1,3420,188,2,11,0,1,3,0,3,1,2,63,1,1,1,2,5,51,1,25,80,16,8,1,2,2,2,2,2,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,2,9993,0.0,2,5,1,2,2,3,11,1,4.0,1,2,0,0,2,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,NaN,1,9,23,5,16,1,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,9993,2,2,2,6,12,2,"1,03851","1,01261"
2,3420,200,3,26,89,17,1,0,5,1,2,56,1,1,2,2,5,50,51,18,51,7,55,1,2,2,2,2,2,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,2,9997,0.0,2,7,1,2,2,3,13,1,5.0,1,7,0,0,8,96,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,1,9,23,5,15,1,1,0,0,1,6,

Como vemos el conjunto de datos tiene un total de 232 columnas y 10101 observaciones. Vamos a reducir el tamaño del dataframe escogiendo aquellas columnas que se vayan a utilizar para las visualizaciones del ejercicio. Se van a seleccionar columnas (un total de 7) relacionadas con el pefil politico del encuestado y variables de idiologia y voyo (2019 y 2023).

In [33]:
columnas = ['PROV','SEXO','EDAD',
            'RECUERDO1R', 'ESCIDEOL',
            'RECUVOTOGR','NIVELESTENTREV']

df = df[columnas]

print(df.shape)

(10101, 7)


Antes de comenzar con las visualizaciones, se creará un diccionario con los colores distintivos (en elc aso de EH Bildu se ha optado por el verde más oscuro de su logo ya que el verde claro o el azuul corporativo podrían confundirse en las visualizaciones con los colores de Vox y Junts respecticamente) de cada uno de los partidos políticos más votados en las últimas elecciones y partidos de segundo orden que podrían salir en las próximas visualizaciones.

In [34]:
color_partidos = {
    'PSOE': '#e4040c',
    'PP': '#0554a3',
    'VOX': '#5cc434',
    'Sumar': '#e40c54 ',
    'Junts': '#04c4b4',
    'ERC': '#fbb31b ',
    'EAJ-PNV': '#048344 ',
    'EH Bildu': '#aabe0f',
    'Otros partidos': 'grey',
    'Ciudadanos': '#fc4c04',
    'JxCat': '#c60f53',
    'Unidas Podemos':  '#7b2c64',
    'Más País':  '#044130' ,
    'N.P.': 'yellow',
    'No votó': 'yellow'
}

Seguidamente se van a llevar a cabo varias transforamciones en dos variables muy importantes de este conjunto de datos que van a ser utilizadas a posteriori. La primera variable que se va a tratar es RECUVOTOGR, la cual contiene el recuerdo del partido votado en las generales del 2019 de las personas encuenstadas.

Siguiendo el libro de codificación, lo primero de todo va a ser nombrar las categorías con el partido político (u opción de voto) correspondiente. Seguidamente se van a concentrar las categorías de partidos minoritarios en 'Otros partidos' dejando a los partidos más votados en aquellas elecciones con una categoría propia, y opciones como 'No recuerda', 'No contesta', 'Nulo', etc., en otras dos categorías. El resultado será la variable RECUVOTOGR_rec.

De igual forma y siguiendo los mismos pasos, se va a recodificar la variable RECURDO1R a RECUERDO!R_rec. Esta variable recodificada de RECVOTOG1 (por el mismo CIS), contiene el voto en las generales de 2023 de las personas encuestadas que dijero que habían votado. El objetivo es aunar un poco más las categorías que deja el CIS.

In [35]:
recodificacion = {
    1: 'PSOE',
    2: 'PP',
    3: 'VOX',
    4: 'Unidas Podemos',
    903: 'En Comú Podem',
    1202: 'En Común-Unidas Podemos',
    5: 'Ciudadanos',
    7: 'Más País',
    901: 'ERC',
    902: 'JxCat',
    904: 'CUP',
    1601: 'EAJ-PNV',
    1602: 'EH Bildu',
    1501: 'Na+',
    501: 'CCa-NC',
    1001: 'Més Compromís',
    1201: 'BNG',
    601: 'PRC',
    202: 'Teruel Existe',
    6: 'PACMA',
    8995: 'Otros partidos',
    8996: 'En blanco',
    9977: 'Voto nulo',
    9998: 'No recuerda',
    9999: 'N.C.',
    0: 'N.P.'
}

df['RECUVOTOGR_rec'] = df['RECUVOTOGR'].map(recodificacion)


recodificacion = {
    'No recuerda': 'N.R + N.C.',
    'N.C.': 'N.R+ N.C.',
    'En blanco': 'Nulo + Blanco',
    'Voto nulo': 'Nulo + Blanco',
    'Na+':'Otros partidos',
    'CCa-NC':'Otros partidos',
    'BNG':'Otros partidos',
    'PRC':'Otros partidos',
    'Teruel Existe':'Otros partidos',
    'PACMA':'Otros partidos',
    'En Comú Podem': 'Unidas Podemos',
    'En Común-Unidas Podemos': 'Unidas Podemos',
    'Més Compromís': 'Más País'
}

df['RECUVOTOGR_rec'] = df['RECUVOTOGR_rec'].replace(recodificacion)


recodificacion_recuerdo1r = {
    2: 'PP',
    1: 'PSOE',
    3: 'VOX',
    21: 'Sumar',
    901: 'ERC',
    902: 'Junts',
    1602: 'EH Bildu',
    1601: 'EAJ-PNV',
    8995: 'Otros partidos',
    9977: 'Nulo + Blanco',
    9997: 'No votó',
    9996: 'Sin condiciones para votar',
    9999: 'N.R + N.C.'
}

df['RECUERDO1R_rec'] = df['RECUERDO1R'].map(recodificacion_recuerdo1r)


A continuación se va a recodificar la variable de nivel de estudios (NIVELESTENTREV) para reducir las categorías que presenta. El objetivo es agrupar categorías en grupos mayores como por ejemplo agrupar todas las formaciones profesionales o los distintos tipos de estudios universitarios y postgraduados 

In [36]:
mapeado = {
    1: 'Menos de 5 años de escolarización',
    2: 'Educación Primaria',
    3: 'F.P. Básica',
    4: 'Educación secundaria',
    5: 'FP de Grado Medio',
    6: 'Bachillerato',
    7: 'FP de Grado Superior',
    8: 'Arquitectura/Ingeniería Técnica',
    9: 'Diplomatura',
    10: 'Grado',
    11: 'Licenciatura',
    12: 'Arquitectura/Ingeniería',
    13: 'Máster oficial universitario',
    14: 'Doctorado',
    15: 'Títulos propios de posgrado',
    16: 'Otros estudios',
    98: 'N.S./No recuerda',
    99: 'N.C.',
    0: 'N.P.'
}

df['NIVELESTENTREV_rec'] = df['NIVELESTENTREV'].map(mapeado)

agrupacion = {
    'FP de Grado Medio': 'Formación Profesional',
    'FP de Grado Superior': 'Formación Profesional',
    'F.P. Básica': 'Formación Profesional',
    'Arquitectura/Ingeniería Técnica': 'Graduado Universitario',
    'Diplomatura': 'Graduado Universitario',
    'Grado': 'Graduado Universitario',
    'Licenciatura': 'Graduado Universitario',
    'Arquitectura/Ingeniería': 'Graduado Universitario',
    'Máster oficial universitario': 'Posgrado Universitario',
    'Títulos propios de posgrado': 'Posgrado Universitario',
    'Doctorado': 'Posgrado Universitario',
}


df['NIVELESTENTREV_rec'] = df['NIVELESTENTREV_rec'].replace(agrupacion)

Se va a configurar un diccionario con las provincias según el libro de código del estudio que proporciona el CIS.

In [37]:
provincia_mapa = {
    1: 'Araba/Álava', 2: 'Albacete', 3: 'Alicante/Alacant', 4: 'Almería', 33: 'Asturias',
    5: 'Ávila', 6: 'Badajoz', 7: 'Balears (Illes)', 8: 'Barcelona', 9: 'Burgos',
    10: 'Cáceres', 11: 'Cádiz', 39: 'Cantabria', 12: 'Castellón/Castelló', 13: 'Ciudad Real',
    14: 'Córdoba', 15: 'Coruña (A)', 16: 'Cuenca', 17: 'Girona', 18: 'Granada',
    19: 'Guadalajara', 20: 'Gipuzkoa', 21: 'Huelva', 22: 'Huesca', 23: 'Jaén',
    24: 'León', 25: 'Lleida', 27: 'Lugo', 28: 'Madrid', 29: 'Málaga',
    30: 'Murcia', 31: 'Navarra', 32: 'Ourense', 34: 'Palencia', 35: 'Palmas (Las)',
    36: 'Pontevedra', 26: 'Rioja (La)', 37: 'Salamanca', 38: 'Santa Cruz de Tenerife',
    40: 'Segovia', 41: 'Sevilla', 42: 'Soria', 43: 'Tarragona', 44: 'Teruel',
    45: 'Toledo', 46: 'Valencia/València', 47: 'Valladolid', 48: 'Bizkaia', 49: 'Zamora',
    50: 'Zaragoza', 51: 'Ceuta', 52: 'Melilla'
}

### 2. Diagrama de Sankey (de flujo)

Todos los análisis de votaciones de las últimas elecciones generales en España que ha habido un flujo en el cambio de voto desde 2019 a 2023 en ciertos partidos. A continuación se va a llevar a cabo la visualziación de un diagrama de Sankey el cual va a permitir ver el flujo en el cambio de voto entre partidos de las personas que fueron encuestadas por el CIS.

Lo primero que se va a ahcer es agrupar las dos variables correspondientes al voto, 'RECUVOTOGR_rec', 'RECUERDO1R_rec'. Después se van a filtrar aquellas categorías minoritarias para que la visualización sea más clara, dejando los partidos principales y algunos de segundo orden no tratados hasta ahora. El siguiente paso será crear dos listas de etiquetas únicas con los aprtidos de inicio (RECUVOTOGR_rec) y los partidos destino (RECUERDO1R_rec) para combinarlas en una sola. El sigiente pase es mapear los nombres de los partidos para crear las listas que van a representar el flujo del voto y la cantidad de este flujo. Su utilizará 'color_partidos' para colorear las etiquetas y finalmente se creará  el diaggrama con plotly especificando las propiedades y diseño adecuados.

In [38]:
recuento_flujo = df.groupby(['RECUVOTOGR_rec', 'RECUERDO1R_rec']).size().reset_index(name='count')

recuento_flujo = recuento_flujo[~recuento_flujo['RECUVOTOGR_rec'].isin(['Nulo + Blanco', 
                                                                        'N.R + N.C.', 
                                                                        'N.R+ N.C.', 
                                                                        'Otros partidos', 
                                                                        'CUP'])]
recuento_flujo = recuento_flujo[~recuento_flujo['RECUERDO1R_rec'].isin(['N.R + N.C.', 
                                                                        'Nulo + Blanco', 
                                                                        'Otros partidos'])]

In [39]:
recuento_flujo

,RECUVOTOGR_rec,RECUERDO1R_rec,count
8,Ciudadanos,No votó,57
11,Ciudadanos,PP,323
12,Ciudadanos,PSOE,52
13,Ciudadanos,Sumar,18
14,Ciudadanos,VOX,39
...,...,...,...
138,VOX,No votó,34
141,VOX,PP,91
142,VOX,PSOE,4
143,VOX,Sumar,3


In [40]:
source_etiquetas = recuento_flujo['RECUVOTOGR_rec'].unique().tolist() # Sacamos partidos de salida
target_etiquetas = recuento_flujo['RECUERDO1R_rec'].unique().tolist() # Sacamos partidos finales
etiquetas_completo = source_etiquetas + target_etiquetas

In [41]:
etiquetas_completo

['Ciudadanos',
 'EAJ-PNV',
 'EH Bildu',
 'ERC',
 'JxCat',
 'Más País',
 'N.P.',
 'PP',
 'PSOE',
 'Unidas Podemos',
 'VOX',
 'No votó',
 'PP',
 'PSOE',
 'Sumar',
 'VOX',
 'EAJ-PNV',
 'EH Bildu',
 'ERC',
 'Junts']

In [46]:
source_indices = {label:idx for idx, label in enumerate(source_etiquetas)}
target_indices = {label:idx + len(source_etiquetas) for idx, label in enumerate(target_etiquetas)}

sources = recuento_flujo['RECUVOTOGR_rec'].map(source_indices).tolist()
targets = recuento_flujo['RECUERDO1R_rec'].map(target_indices).tolist()
valores = recuento_flujo['count'].tolist()

nodo_colores = [color_partidos[label] for label in etiquetas_completo]

In [ ]:
fig = go.Figure(go.Sankey( # Construye una figura con un grafico de tipo Sankey 
    # Configuración de los nodos (cajones de cada lado)
    node=dict( 
        pad=5, # Espacio vertical entre nodos de la misma columna (px)
        thickness =15, # Grosor de cada nodo (px)
        line = dict(color='black', width=0.5), # Borde de los nodos
        label = etiquetas_completo,  #Lista de textos de los nodos (orden = índice 0..N-1)
        color = nodo_colores # Lista de colores, uno por cada nodo
    ),
    # Configuración de los enlaces (flujos)
    link = dict(
        source=sources, # Índices de nodo origen 
        target=targets, # Índices de nodo destino
        value=valores # Peso de cada enlace 
    )
))

fig.update_layout(title_text = 'Cambio de voto desde las elecciones generales de 2019 a 2023', font_size=12, height = 600)


fig.show()